<a href="https://colab.research.google.com/github/aryajpandey/Solar-Roof-Optimization-Solver/blob/main/Solar_optimization_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomo
!apt-get install -y coinor-cbc glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5 libamd2
  libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 11 newly installed, 0 to remove and 30 not upgraded.
Need to get 3,533 kB of archives.
After this operation, 10.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5

In [ ]:

import pyomo.environ as pyo


model = pyo.ConcreteModel()

# -------------------------------------------------------------------------
# Data Definition
# -------------------------------------------------------------------------
# Assuming the roof is subdivided into 8 segments, each with:
# - usable_area[i]: how many sq ft can hold panels
# - shading_factor[i]: fraction of unshaded irradiance
# - structural_limit[i]: how many panels can be placed in that segment
#   (due to load constraints)

segments = range(1, 9)

# Usable roof area for each of the 8 segments (in sq. ft).
usable_area = {
    1: 2292.42,
    2: 359.55,
    3: 128,
    4: 292.21,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}

# Shading factor for each segment: 1.0 = no shade, 0.8 = some shade, etc.
# We'll assume the building is historically overshadowed in a few areas,
# and not overshadowed in others.
shading_factor = {
    1: 0.45,
    2: 0.2,
    3: 0.85,
    4: 0.80,
    5: 1.0,
    6: 1.0,
    7: 1.0,
    8: 1.0,

}

# Structural or local code constraints might cap the # of panels each segment can handle
# e.g., due to weight restrictions. We'll guess for demonstration.
max_panels_per_segment = {
    1: 20,
    2: 100,
    3: 120,
    4: 60,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}

# We also know:
panel_area    = 17.5      # sq. ft per panel (approx for 320 W panel)
panel_power   = 320       # W (DC rating)
panel_cost    = 400       # $ per panel (hardware + labor)
performance_ratio = 0.80  # Inverter & system losses

# Typical Chicago sun hours:
peak_sun_hours = 4.5
days_per_year  = 365
annual_hours   = peak_sun_hours * days_per_year

# Project-wide constraints
budget       = 350000  # $ total
capacity_kW  = 500     # 500 kW capacity limit (in W => 500,000 W)

# We'll convert capacity_kW to W for consistency
capacity_limit = capacity_kW * 1000  # 500,000 W

# -------------------------------------------------------------------------
# Define Model Variables
# -------------------------------------------------------------------------
# x[i] = number of panels installed in segment i

model.x = pyo.Var(segments, domain=pyo.NonNegativeIntegers)

# -------------------------------------------------------------------------
# Objective Function
# Maximizing total annual energy (kWh)
# -------------------------------------------------------------------------


def total_energy_rule(m):
    # For each panel on segment i,
    #   output_i = panel_power(W) * shading_factor[i] * performance_ratio * annual_hours / 1000
    return sum(
        m.x[i] * (
            panel_power
            * shading_factor[i]
            * performance_ratio
            * annual_hours
            / 1000.0  # convert Wh -> kWh
        )

        for i in segments

    )

model.total_energy = pyo.Objective(rule=total_energy_rule, sense=pyo.maximize)



# -------------------------------------------------------------------------
# Constraints
# -------------------------------------------------------------------------


# Area Constraint
#    For each segment i, the total panel area can't exceed the usable area.
#    i.e. x[i] * panel_area <= usable_area[i]

def area_rule(m, i):
    return m.x[i] * panel_area <= usable_area[i]

model.area_constraint = pyo.Constraint(segments, rule=area_rule)


# Budget Constraint
def budget_rule(m):
    return sum(m.x[i] * panel_cost for i in segments) <= budget

model.budget_constraint = pyo.Constraint(rule=budget_rule)


# Capacity Constraint (to remain under 500 kW for better incentives)
def capacity_rule(m):
    return sum(m.x[i] * panel_power for i in segments) <= capacity_limit

model.capacity_constraint = pyo.Constraint(rule=capacity_rule)


# Structural / Max Panels per Segment

# Some segments may have an upper bound for # of panels based on load
def structural_rule(m, i):
    return m.x[i] <= max_panels_per_segment[i]

model.struct_rule = pyo.Constraint(segments, rule=structural_rule)

# -------------------------------------------------------------------------
# Solver
# -------------------------------------------------------------------------
# For Google Colab, you may need:
# !apt-get install -y coinor-cbc or glpk
# !pip install pyomo

solver = pyo.SolverFactory('cbc')  # can also be either 'glpk', 'gurobi', 'neos-cbc', etc. incase cbc doesn't work
results = solver.solve(model, tee=True)

# -------------------------------------------------------------------------
# Output
# -------------------------------------------------------------------------
print("Solver Status   :", results.solver.status)
print("Termination Cond:", results.solver.termination_condition)

optimal_energy = pyo.value(model.total_energy)
print(f"\nMaximized Annual Energy (kWh): {optimal_energy:,.0f}")

# Summarize how many panels are allocated per segment
total_panels = 0
for i in segments:
    xi = pyo.value(model.x[i])
    total_panels += xi
    seg_area = usable_area[i]
    print(f"Segment {i}: {xi} panels   [Usable area: {seg_area} ft^2]")

print(f"\nTotal panels installed: {total_panels}")

# -------------------------------------------------------------------------
# Basic Economic Calculation
# -------------------------------------------------------------------------
# Computing approximate total cost, capacity (kW), and payback

total_cost = total_panels * panel_cost
capacity_installed_W = sum( (pyo.value(model.x[i]) * panel_power) for i in segments )
annual_kWh = optimal_energy

# Federal ITC (26% for example)
itc_amount = 0.26 * total_cost
net_install_cost = total_cost - itc_amount

# Assume we offset electricity at $0.13 / kWh + an REC incentive of $0.045 for 15 years
energy_savings = annual_kWh * 0.13
rec_incentive  = annual_kWh * 0.045  # for 15 years (not discounted here)
first_year_cashflow = energy_savings + rec_incentive

print("\n--- Economic Overview ---")
print(f"Gross System Cost:          ${total_cost:,.0f}")
print(f"Federal ITC (26%):          ${itc_amount:,.0f}")
print(f"Net Installation Cost:      ${net_install_cost:,.0f}")
print(f"Installed Capacity (kW):    {capacity_installed_W/1000:.1f} kW")
print(f"Annual Production (kWh):    {annual_kWh:,.0f}")
print(f"First-Year Electricity $:   ${energy_savings:,.0f}")
print(f"First-Year REC Incentives:  ${rec_incentive:,.0f} (for 15 years)")
print(f"---------------------------")
simple_payback = net_install_cost / (energy_savings + rec_incentive)
print(f"Estimated Simple Payback:   {simple_payback:.2f} years")
print("Done.")


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpx1mseut9.pyomo.lp -stat=1 -solve -solu /tmp/tmpx1mseut9.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 0 (-18) rows, 0 (-8) columns and 0 (-32) elements
Statistics for presolved model
Original problem has 8 integers (0 of which binary)


Problem has 0 rows, 0 columns (0 with objective) and 0 elements
There are 22230 singletons with no objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G oth